In [3]:
import hashlib
import json
import os
from pathlib import Path
import subprocess

In [4]:
BASE_DIR = Path().resolve().parent

In [5]:
transform_script = BASE_DIR / "etl" / "transform" / "vendas_transform" / "tratar_vendas.py"
load_script = BASE_DIR / "etl" / "load" / "vendas_load" / "load_vendas.py"

In [6]:
RAW_FODLER = Path("../data/raw")
HASH_TRACKER = Path("config/hash_tracker.json")

In [7]:
def calcular_hash_md5(caminho_arquivo):
    with open(caminho_arquivo, "rb") as f:
        return hashlib.md5(f.read()).hexdigest()
    
def carregar_hashes_anteriores():
    if not HASH_TRACKER.exists():
        return {}
    with open(HASH_TRACKER, "r") as f:
        return json.load(f)
    
def salvar_hashes_atuais(hashes):
    HASH_TRACKER.parent.mkdir(parents=True, exist_ok=True)
    with open(HASH_TRACKER, "w") as f:
        json.dump(hashes, f, indent=2)

def encontrar_csvs(pasta_base):
    return list(pasta_base.rglob("*.csv"))

def cliente_do_arquivo(caminho_arquivo):
    """
    Extrai o nome do cliente com base no padrão: data/raw/{cliente}_raw
    """

    partes = Path(caminho_arquivo).parts
    try:
        idx_raw = partes.index("raw")
        pasta_cliente = partes[idx_raw - 1]
        return pasta_cliente.replace("_raw", "")
    except:
        return "nome desconhecido"

def main():
    arquivos = encontrar_csvs(RAW_FODLER)
    hashes_anteriores = carregar_hashes_anteriores()
    novos_hashes = {}

    arquivos_alterados = []


    for arquivo in arquivos:
        hash_atual = calcular_hash_md5(arquivo)
        caminho_str = str(arquivo)

        if hashes_anteriores.get(caminho_str) != hash_atual:
            print(f"🟢 Alteração detectada: {caminho_str}")
            arquivos_alterados.append(caminho_str)            

        novos_hashes[caminho_str] = hash_atual

    if arquivos_alterados:

        subprocess.run(["python", str(transform_script)], check=True)
        subprocess.run(["python", str(load_script)], check=True)


        print("🔄 Arquivos alterados:")
        for arquivo in arquivos_alterados:
            print(f" - {arquivo}")
        print("🚀 Executar pipeline para arquivos modificados!")
    else:
        print("🟡 Nenhuma modificação encontrada.")

    salvar_hashes_atuais(novos_hashes)
        

In [8]:
if __name__ == "__main__":
    main()
   

🟢 Alteração detectada: ../data/raw/vendas_raw/base_vendas_supermercado_suja.csv
📥 Linhas novas detectadas: 0
✅ Nenhuma nova linha para inserir.
🔄 Arquivos alterados:
 - ../data/raw/vendas_raw/base_vendas_supermercado_suja.csv
🚀 Executar pipeline para arquivos modificados!
